# CSE-CIC-IDS 2017

In [1]:
model_id = "gradientboostedtrees3-gbt-multiclass"

In [2]:
import numpy as np
np.random.seed(42)
np.set_printoptions(suppress=True)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import glob, pickle, time, sklearn

In [3]:
print(sklearn.__version__)

0.21.3


In [4]:
NOTEBOOK_PATH = "C:/Users/Xetrov/Desktop/SciFair20/Code/"

In [5]:
x_scaled = pd.read_csv(NOTEBOOK_PATH + "IDS2017/x_scaled_powertransform.csv")

y_df_enc = pd.read_csv(NOTEBOOK_PATH + "IDS2017/y_grouped_1henc.csv")

# Split data into train and test

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
x_train, x_valtest, y_train, y_valtest = train_test_split(x_scaled, y_df_enc, test_size = 0.4, random_state = 42)

In [8]:
x_val, x_test, y_val, y_test = train_test_split(x_valtest, y_valtest, test_size = 0.5, random_state = 42)

In [9]:
del x_train
del y_train

del x_valtest 
del y_valtest

# ADASYN

In [10]:
x_train_res = pd.read_csv(NOTEBOOK_PATH + "IDS2017/x_adasyn.csv").to_numpy()

In [11]:
y_train_res = pd.read_csv(NOTEBOOK_PATH + "IDS2017/y_adasyn.csv")

In [12]:
y_train_res = y_train_res[y_train_res.columns[0]].to_numpy()

# Train model

In [13]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier

In [14]:
clf = GradientBoostingClassifier(n_estimators=250, max_depth=9, subsample=0.8, max_features="sqrt", random_state=42, verbose=2)

In [ ]:
%%time
clf.fit(x_train_res, y_train_res)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1    10833067.2170     1008090.3678          816.18m
         2     8591089.3198      560345.8347         1465.28m
         3     6972709.7357      404523.7588         1778.71m


**Save model**

In [42]:
import pickle

In [43]:
import time

model_filename = model_id  + " [" + time.strftime("%Y%m%d %H%M") + "]"

In [54]:
save_file = open(NOTEBOOK_PATH + "Models/" + model_filename + ".pkl", "wb")
saved_model = pickle.dump(clf2, save_file)
save_file.close()

**Test model**

In [21]:
pred = clf.predict(x_val)

In [22]:
pred_index = pred
y_index = list(np.argmax(y_val.to_numpy(), axis=1))

atktypes = sorted(y_val.columns)
pred_series = pd.Series(pred_index, name="Pred").replace(dict(enumerate(atktypes)))
y_series = pd.Series(y_index, name="Actual").replace(dict(enumerate(atktypes)))

matrix = pd.crosstab(pred_series, y_series, margins=True)
matrix

Actual,BENIGN,Botnet,Brute Force,DoS/DDoS,Infiltration,PortScan,Web Attack,All
Pred,,,,,,,,
BENIGN,434267,28,0,380,0,82,15,434772
Botnet,3972,369,0,0,0,0,0,4341
Brute Force,774,0,2810,0,0,58,0,3642
DoS/DDoS,8136,0,0,75953,0,13,0,84102
Infiltration,18,0,0,0,7,0,0,25
PortScan,6616,0,0,2,0,31632,0,38250
Web Attack,424,0,0,175,0,2,416,1017
All,454207,397,2810,76510,7,31787,431,566149


# F1 Score

In [25]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [34]:
pred = clf.predict(x_test)
pred_index = list(pred)

In [35]:
y_index = list(np.argmax(y_test.to_numpy(), axis=1))
f1_micro = f1_score(y_index, pred_index, average='micro')
f1_macro = f1_score(y_index, pred_index, average='macro')

print(f1_micro)
print(f1_macro)

0.9255814282105947
0.5620636156417309


In [19]:
#prev


0.9634371870302694
0.676572196968186


Prev scores

0.9209501385677622

0.5228619390734598

In [20]:
atktypes = sorted(y_val.columns)
pred_series = pd.Series(pred_index, name="Pred").replace(dict(enumerate(atktypes)))
y_series = pd.Series(y_index, name="Actual").replace(dict(enumerate(atktypes)))

matrix = pd.crosstab(pred_series, y_series, margins=True)
# matrix.to_csv(NOTEBOOK_PATH + "Confusion Matrices/test_" + model_filename + ".csv")
matrix

Actual,BENIGN,Botnet,Brute Force,DoS/DDoS,Infiltration,PortScan,Web Attack,All
Pred,,,,,,,,
BENIGN,434869,30,0,407,0,84,13,435403
Botnet,3881,354,0,0,0,0,0,4235
Brute Force,777,0,2752,0,0,56,0,3585
DoS/DDoS,8299,0,0,75311,0,9,0,83619
Infiltration,21,0,0,1,5,0,0,27
PortScan,6494,0,0,3,0,31726,0,38223
Web Attack,449,0,0,175,0,1,432,1057
All,454790,384,2752,75897,5,31876,445,566149


In [20]:
# Prev

Actual,BENIGN,Botnet,Brute Force,DoS/DDoS,Infiltration,PortScan,Web Attack,All
Pred,,,,,,,,
BENIGN,412000,143,0,814,1,572,16,413546
Botnet,15925,240,0,0,0,0,0,16165
Brute Force,2230,0,2752,0,0,56,0,5038
DoS/DDoS,8478,0,0,74865,0,23,4,83370
Infiltration,2802,1,0,9,4,57,0,2873
PortScan,1233,0,0,31,0,31109,0,32373
Web Attack,12122,0,0,178,0,59,425,12784
All,454790,384,2752,75897,5,31876,445,566149


# Log results

In [72]:
model_log = open(NOTEBOOK_PATH + "model_log.txt", "a")

model_log.write("\n" + model_filename)
model_log.write("\n\tF1 Micro: " + str(f1_micro))
model_log.write("\n\tF1 Macro: " + str(f1_macro))

model_log.close()